In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ],
)

model = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key="not_required",
    model="qwen2.5-32b",
    temperature=0
)

output_parser = StrOutputParser()

In [3]:
prompt_value = prompt.invoke({"dish": "カレー"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)

print(output)

NotFoundError: Error code: 404 - {'error': {'message': 'model "hf.co/Qwen/Qwen2.5-32B-Instruct-GGUF:Q5_K_M" not found, try pulling it first', 'type': 'api_error', 'param': None, 'code': None}}

In [11]:
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "カレー"}):
    print(chunk, end="", flush=True)

材料（4人分）
・玉ねぎ　1個
・にんじん　1本
・じゃがいも　2個
・鶏肉　400g
・水　800ml
・カレールウ　4～5表匙
・塩　小さじ1
・こしょう　少々
・植物油　大さじ2

作り方
１、玉ねぎを薄切りにして、にんじんとじゃがいもは皮を剥いて厚手で切ります。
２、フライパンに油を入れ、火を点けて強めの中火で加熱します。油が熱くなったら、鶏肉を入れて色づくまで焼きます。
３、鶏肉に色がついたら、玉ねぎ・にんじん・じゃがいもを加えてさらに2～3分炒めます。
４、水を入れ、火を弱くして20分ほど煮込みます。野菜が柔らかくなったら、カレールウを入れて1分ほどかき混ぜます。
５、塩とこしょうで味を調えて完成です。

In [12]:
chain = prompt | model | output_parser

outputs = chain.batch([{"dish": "カレー"}, {"dish": "寿司"}])
print(outputs)

['材料（4人分）\n・玉ねぎ\u30001個\n・にんじん\u30001本\n・じゃがいも\u30002個\n・鶏肉\u3000400g\n・水\u3000800ml\n・カレールウ\u30004～5表匙\n・塩\u3000小さじ1\n・こしょう\u3000少々\n・植物油\u3000大さじ2\n\n作り方\n１、玉ねぎを薄切りにして、にんじんとじゃがいもは皮を剥いて厚手で切ります。\n２、フライパンに油を入れ、火を点けて強めの中火で加熱します。油が熱くなったら、鶏肉を入れて色づくまで焼きます。\n３、鶏肉に色がついたら、玉ねぎ・にんじん・じゃがいもを加えてさらに2～3分炒めます。\n４、水を入れ、火を弱くして20分ほど煮込みます。野菜が柔らかくなったら、カレールウを入れて1分ほどかき混ぜます。\n５、塩とこしょうで味を調えて完成です。', '材料：\n\n・酢飯用短粒米\u30003合\n・りんご酢\u3000大さじ2\n・砂糖\u3000小さじ1\n・塩\u3000少々\n・水\u3000100ml\n\n作り方：\n１．酢飯の準備\n\u3000米を洗い、30分ほど浸けておく。\n\u3000鍋に水を入れて沸騰したら、米を入れて弱火で20分ほど炊く。\n\u3000できたら、ふたをして5分ほど蒸らす。\n２．酢の準備\n\u3000りんご酢、砂糖、塩、水を混ぜてよくかき回す。\n３．酢飯の作り方\n\u3000炊いたご飯を大きな容器に入れ、酢をふんだんにかける。\n\u3000しきりに切ないで、扇風機で冷やす。\n４．ネタの準備\n\u3000好みのネタ（たまご、イクラ、サーモンなど）を用意する。\n５．握り方\n\u3000手の平に少量の酢飯をのせ、中央にネタを置く。\n\u3000指で軽く押さえながら、ねじって形を作る。\n\n注意点：\n・米を炊くときは、水分を調節することが重要です。\n・酢飯を冷やすときは、扇風機を使うと良いでしょう。']


In [13]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください。"),
        ("human", "{question}"),
    ],
)

cot_chain = cot_prompt | model | output_parser

In [14]:
summerize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください。"),
        ("human", "{text}"),
    ],
)

summerize_chain = summerize_prompt | model | output_parser

In [17]:
cot_summerize_chain = cot_chain | summerize_chain
cot_summerize_chain.invoke({"question": "10 + 2 * 3"})

'最終的な答えは16です。'

In [18]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。"),
        ("human", "{topic}"),
    ],
)
optimistic_chain = optimistic_prompt | model | output_parser

In [19]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。"),
        ("human", "{topic}"),
    ],
)
pessimistic_chain = pessimistic_prompt | model | output_parser

In [20]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "生成AIの進化について"})
pprint.pprint(output)

{'optimistic_opinion': '生成AIの進化は、人類の歴史において最も革新的な技術的進歩の一つです。この分野の急速な発展は、人間の創造性とイノベーションの限界を拡大し、新たな可能性を切り開いています。生成AIは、画像、動画、音楽、文章など、多様なメディア形式で、高品質のコンテンツを作成する能力を持っています。これにより、芸術、エンターテイメント、教育、広告などの分野で新たな表現方法やビジネスモデルが生まれます。\n'
                       '\n'
                       'さらに、生成AIは、データ分析やプログラミングのような複雑なタスクを自動化することで、ビジネスプロセスの効率化にも貢献しています。企業は、従来人力で行っていた作業をAIに任せることで、コスト削減と生産性の向上を実現できます。\n'
                       '\n'
                       '生成AIの進化は、社会全体への影響も考慮する必要があります。雇用市場への影響や、偽情報の拡散防止などの倫理的な問題について、慎重な議論と対策が求められます。しかし、これらの課題に対処することで、生成AIの利点を最大化し、社会全体の発展に貢献できるでしょう。\n'
                       '\n'
                       '結局、生成AIの進化は、私たちが生きる世界をより豊かで多様なものにする可能性を持っています。楽観的に見て、この技術の将来性は非常に明るく、人類の創造性とイノベーションをさらに促進するでしょう。',
 'pessimistic_opinion': '生成AIの進化は、人間の創造性や労働の価値を奪うだけだ。AIがどんどん高度になるにつれて、人間の仕事は機械に取って代わられていく。芸術、音楽、文学など、創造性を必要とする分野でも、AIが人間を上回る日は近い。人々は自分の才能や努力が不要と感じ、自信を失っていくだろう。\n'
                        '\n'
                        'さらに、生成AIの進化は、情報の信頼性や真実性の低下につながる。偽物のニュースや画像、動画が大量に生産され、人々は何が本

In [22]:
synthesis_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。2つの意見をまとめてください。"),
        ("human", "楽観的意見: {optimistic_opinion}\n悲観的意見: {pessimistic_opinion}"),
    ],
)

synthesis_chian = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesis_prompt 
    | model 
    | output_parser
)

In [23]:
output = synthesis_chian.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化は、人類の歴史において最も革新的な技術的進歩の一つであり、新たな可能性を切り開いています。一方で、雇用市場への影響や偽情報の拡散防止などの倫理的な問題について慎重な議論と対策が必要です。生成AIは、高品質のコンテンツを作成する能力を持っており、芸術、エンターテイメント、教育、広告などの分野で新たな表現方法やビジネスモデルを生み出します。また、データ分析やプログラミングのような複雑なタスクを自動化することで、ビジネスプロセスの効率化にも貢献しています。ただし、生成AIの進化は人間の創造性や労働の価値を奪う可能性もあり、情報の信頼性や真実性の低下につながる危険性があります。社会全体として見ても、既存の社会構造や価値観を崩壊させる可能性があるため、私たちは人間の価値観や社会の未来について真剣に考える必要があります。
